In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from tqdm.notebook import tqdm

import pandas as pd
import time
import re
import csv
import datetime

print('search')
searchQ = input()
print('startYear')
startYear = input()
print('endYear')
endYear = input()
print('wantPage')
want_page = int(input())

print("start crawling..")

driver = webdriver.Chrome()
#xpath = driver.find_element_by_xpath

#검색
driver.get('https://www.dbpia.co.kr/')
keyword = driver.find_element("id",'searchInput')
keyword.clear()
keyword.send_keys(searchQ)
serach_click_btn = driver.find_element(By.XPATH,'//*[@id="submitSearchInput"]/span')
driver.execute_script("arguments[0].click();",serach_click_btn)

#날짜 지정 및 텍스트 박스 설정
time.sleep(10)
driver.find_element(By.XPATH,'//*[@id="yearRangeMin"]').clear()
driver.find_element(By.XPATH,'//*[@id="yearRangeMin"]').send_keys(startYear)
driver.find_element(By.XPATH,'//*[@id="yearRangeMax"]').clear()
driver.find_element(By.XPATH,'//*[@id="yearRangeMax"]').send_keys(endYear)
click_btn = driver.find_element(By.XPATH,'//*[@id="yearRangeSearch"]')
driver.execute_script("arguments[0].click();",click_btn)

title_list = []
author_list = []
publisher_list = []
journal_list = []
volume_list = []
date_list = []
abstract_list = []
link_list = []

list_count = 1

while(True) :
    if(list_count > want_page):
        break
    else:
        try:
            if list_count == 1:
                items_source = driver.page_source
                bs = BeautifulSoup(items_source, 'html.parser')
                items = bs.find_all('article', {'class':'thesis__summary'})
                count = len(items)
                list_count = list_count + 1
                
            else:
                time.sleep(10)
                next_click_btn = driver.find_element(By.XPATH,f'//*[@id="pageList"]/a[{list_count}]')
                driver.execute_script("arguments[0].click();",next_click_btn)
                
                time.sleep(5)
                print(f'this page {list_count}')
                items_source = driver.page_source
                bs = BeautifulSoup(items_source, 'html.parser')
                items = bs.find_all('article', {'class':'thesis__summary'})
                count = len(items)
                list_count = list_count + 1
                
        except Exception as e:
            print(f"error: {e}")
            break
            
    for item in tqdm(items):
        # 제목
        try:
            title = ''
            title = item.find('h2',{'class':'thesis__tit'}).text
        except:
            title = ''
    
        #저자
        try:
            author = ''
            author = item.find('span',{'class':'thesis__author'}).text
        except:
            author = ''
        
        # 퍼블리셔 
        try:
            publisher = ''
            publisher = item.find('section',{'class':'thesisAdditionalInfo thesis__info'}).find('a',{'class':'nodeIprd thesis__item'}).text
        except:
            publisher = ''
    
        # 저널명
        try:
            journal = ''
            journal = item.find('section',{'class':'thesis__info'}).find('span',{'class':'thesis__item'}).text
        except:
            journal = ''
    
        #볼륨
        try:
            volume = ''
            volume = item.find('section',{'class':'thesisAdditionalInfo thesis__info'}).find('a',{'class':'nodeVois thesis__item'}).text
        except:
            volume = ''
    
        #날짜
        try:
            date = ''
            date = item.find('section',{'class':'thesisAdditionalInfo thesis__info'}).find('span',{'class':'thesis__item'}).text
        except:
            date = ''
    
        #초록
        try:
            abstract = ''
            abstract = item.find('span',{'class':'thesis__abstract'}).text
        except:
            abstract = ''
    
        #link
        try:
            link = ''
            link = item.find('a',{'class':'thesis__link title'}).attrs['href']
        except:
            link = ''
    
        title_list.append(title)
        author_list.append(author)
        publisher_list.append(publisher)
        link_list.append('https://www.dbpia.co.kr'+link)
        journal_list.append(journal)
        volume_list.append(volume)
        date_list.append(date)
        abstract_list.append(abstract)

result = pd.DataFrame({'title_list':title_list, 'author_list':author_list, 'publisher_list':publisher_list, 'link_list':link_list,'journal_list':journal_list, 'volume_list': volume_list, 'date_list':date_list, 'abstract_list': abstract_list})

search
한국 기준 금리
startYear
2000
endYear
2023
wantPage
5
start crawling..


  0%|          | 0/20 [00:00<?, ?it/s]

this page 2


  0%|          | 0/20 [00:00<?, ?it/s]

this page 3


  0%|          | 0/20 [00:00<?, ?it/s]

this page 4


  0%|          | 0/20 [00:00<?, ?it/s]

this page 5


  0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
result = result.drop_duplicates()
result

,title_list,author_list,publisher_list,link_list,journal_list,volume_list,date_list,abstract_list
0,주택가격 버블과 기준금리 변동의 시사점: 한국과 미국의 비교,,한국재무관리학회,https://www.dbpia.co.kr/journal/articleDetail?...,2022.4,39(2),2022.4,본 연구는 미국과 한국 주택가격 버블을 실증적으로 비교 분석하여 최근관심을 끌고 있...
1,한국은행의 기준금리 결정요인 분석 : 물가안정목표제 하의 기준금리 결정 행태,심성보,,https://www.dbpia.co.kr/journal/detail?nodeId=...,2017,,2017,본 연구의 목적은 순서화 된 probit 모형(ordered probit model...
2,한국은행 기준금리 결정에 대한 연구,김양배,,https://www.dbpia.co.kr/journal/detail?nodeId=...,2015,,2015,
3,기준금리 인상이 금융안정에 미치는 영향 분석,노유철,한국은행,https://www.dbpia.co.kr/journal/articleDetail?...,2022.12,,2022.12,
4,기준금리 변동이 주식수익률에 미치는 영향에 관한 연구,,한국무역보험학회,https://www.dbpia.co.kr/journal/articleDetail?...,2022.10,23(5),2022.10,본 연구는 기준금리 변경은 주식수익률에 영향을 주므로 기준금리 변경일 기준으로 일정...
...,...,...,...,...,...,...,...,...
95,금리 스프레드의 경제 예측력에 관한 비교 연구: 스프레드 구간과 금융위기 전후 분석...,,한국기업경영학회,https://www.dbpia.co.kr/journal/articleDetail?...,2020.1,27(2),2020.1,본 연구는 우리나라의 최근 자료를 이용하여 금리 스프레드의 실물경제 및 금융시장 예...
96,한국 신용부도스왑(CDS) 스프레드의 결정요인,,한국산업경제학회,https://www.dbpia.co.kr/journal/articleDetail?...,2011.12,24(6),2011.12,본 연구는 2007년 1월부터 2009년 9월까지 신용구조모형 가설에 따라 한국의 ...
97,IFRS 17과 SolvencyⅡ의 비교 · 분석을 통한 보험사의 재무건전성 감독기...,,한국국제회계학회,https://www.dbpia.co.kr/journal/articleDetail?...,2019.4,84,2019.4,국제회계기준위원회(IASB)는 보험계약에 대한 일관된 회계처리를 목적으로 보험부채의...
98,비은행금융기관들의 가중평균금리가 비은행금융기관들의 여신과 수신에 미치는 영향에 관한 연구,,한국신용카드학회,https://www.dbpia.co.kr/journal/articleDetail?...,2019.1,13(4),2019.1,본 논문은 비은행금융기관들의 가중평균금리가 비은행금융기관들의 여신과 수신에 미치는 ...


In [8]:
result.to_excel(f'{searchQ}_crawling_data.xlsx', index=False)